In [1]:
import pandas as pd
# import scispacy
import spacy
import re
import nltk
from spacy import displacy
from spacy.matcher import Matcher
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
med_transcript = pd.read_csv("mtsamples.csv", index_col=0)

In [3]:
med_transcript.info()
med_transcript.head(1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4999 entries, 0 to 4998
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   description        4999 non-null   object
 1   medical_specialty  4999 non-null   object
 2   sample_name        4999 non-null   object
 3   transcription      4966 non-null   object
 4   keywords           3931 non-null   object
dtypes: object(5)
memory usage: 234.3+ KB


,description,medical_specialty,sample_name,transcription,keywords
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."


In [4]:
# Drug Detection Model
nlp = spacy.load("en_ner_bc5cdr_md")


c:\Users\RAZAK\Desktop\venv\venv\lib\site-packages\spacy\util.py:833: UserWarning: [W095] Model 'en_ner_bc5cdr_md' (0.4.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
pattern = [{'ENT_TYPE':'CHEMICAL'}, {'LIKE_NUM': False} ,{'IS_ASCII': True}]
matcher = Matcher(nlp.vocab)
matcher.add("DRUG_DOSE", [pattern])

In [6]:
def pdftoword(pdf, pg):
    from io import StringIO
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfdocument import PDFDocument
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.pdfpage import PDFPage
    from pdfminer.pdfparser import PDFParser

    output_string = StringIO()
    with open(pdf, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for i,page in enumerate(PDFPage.create_pages(doc)):
            if i == pg:
                interpreter.process_page(page)
    return output_string.getvalue()

lists = word_tokenize(pdftoword('r.pdf', 0))

new_list_nodates = filter(
    lambda x: not re.match('[0-9]{2}[\/,:][0-9]{2}[\/,:][0-9]{2,4}', x),
    lists)
# print(list)

text = ""
for word in new_list_nodates :
    text += word + ' '
text

'RAZAK Speciality clinic # 11 , guppe road , rvce campus bangalore Doctor name : Razak Date : 22-02-2022 Patient name : Shivnath Age : 69 yrs prescription : Marijuana - 4 times Crocin - twice a day Dolo 650 - 100 times a day Volini – when required price : 1000/- '

In [7]:
nlp_ob = nlp(text)
displacy.render(nlp_ob[:], style='ent', jupyter=True)

In [8]:
pattern = [{'ENT_TYPE':'CHEMICAL'}, {'IS_ASCII': True}]
matcher = Matcher(nlp.vocab)
matcher.add("DRUG_DOSE", [pattern])

In [9]:
'''Drug Name &  Dose Extraction'''
matches = matcher(nlp_ob)
for name in nlp_ob.ents:
    if name.label_ == 'CHEMICAL':
        print(name)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # get string representation
    span = nlp_ob[start:end]  # the matched span
    print(string_id, start, end, span.text) 

Crocin
DRUG_DOSE 37 39 Crocin -


In [10]:
# Person recognition   
nlp5 = spacy.load('en_core_web_sm')

In [11]:
''' Doctors Name Extraction '''
def find_dr(word):
    for i1, w in enumerate(word):
        if (word[i1:i1+3]).lower() == 'dr.' :
            return i1
    return -1

names_doc_tokens = word_tokenize(pdftoword('Sample Patient Profiles and Prescriptions.pdf', 5))
names_doc_text = ""

for token_idx,w in enumerate(names_doc_tokens):
    if (find_dr(w) == 0) and (len(w) <= 15) :
        break

for word in names_doc_tokens :
    names_doc_text += word + ' '

doctor = names_doc_tokens[token_idx]
# print(doctor)
nlp_names = nlp5(names_doc_tokens[token_idx] + ' ' + names_doc_tokens[token_idx + 1] + '  ' + names_doc_tokens[token_idx + 2])
# print(nlp_names)
displacy.render(nlp_names[:], style='ent', jupyter=True)
for name in nlp_names.ents:
    if name.label_ == 'PERSON':
        doctor = doctor + ' ' + str(name)
print(doctor)


Dr. Best


In [12]:
import dateutil.parser
'''Extract Date'''

def find_date(word):
    for i1, w in enumerate(word):
        if (word[i1:i1+4]).lower() == 'date' :
            # print(word)
            return i1
    return -1

dates_tokens = word_tokenize(pdftoword('Sample Patient Profiles and Prescriptions.pdf', 5))
dates_text = ""


for token_idx,w in enumerate(dates_tokens):
    if (find_date(w) != -1):
        break
probable_dates = [dates_tokens[token_idx + 1] , dates_tokens[token_idx + 2]]
# print(dates_tokens)
date = ''
for probable_date in probable_dates:
    try:
        date = dateutil.parser.parse(probable_date).date()
        break
    except:
        pass
print(date)

2013-08-13


In [13]:
import csv

In [14]:
'''Starting Training using uci drug dataset'''

df_uci = pd.read_csv("drugsComTrain_raw.tsv", sep='\t')
df_uci.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [15]:
nlp.pipe_names

['tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer', 'parser', 'ner']

In [16]:
ex = "Nexplanon 650mg Crocin"
displacy.render(nlp(ex)[:], style='ent', jupyter=True)

In [17]:
def process_review(review):
    processed_token = []
    for token in review.split():
        token = ''.join(e.lower() for e in token if e.isalnum())
        processed_token.append(token)
    return ' '.join(processed_token)

In [18]:
process_review("Nexplanon 650mg Crocin")

'nexplanon 650mg crocin'

In [19]:
all_drugs_uci = df_uci['drugName'].unique().tolist()
all_drugs_uci = [x.lower() for x in all_drugs_uci]
all_drugs_uci

['valsartan',
 'guanfacine',
 'lybrel',
 'ortho evra',
 'buprenorphine / naloxone',
 'cialis',
 'levonorgestrel',
 'aripiprazole',
 'keppra',
 'ethinyl estradiol / levonorgestrel',
 'topiramate',
 'l-methylfolate',
 'pentasa',
 'dextromethorphan',
 'nexplanon',
 'liraglutide',
 'trimethoprim',
 'amitriptyline',
 'lamotrigine',
 'nilotinib',
 'atripla',
 'trazodone',
 'etonogestrel',
 'etanercept',
 'tioconazole',
 'azithromycin',
 'eflornithine',
 'daytrana',
 'ativan',
 'imitrex',
 'sertraline',
 'toradol',
 'viberzi',
 'mobic',
 'dulcolax',
 'morphine',
 'moviprep',
 'trilafon',
 'fluconazole',
 'contrave',
 'clonazepam',
 'metaxalone',
 'venlafaxine',
 'ledipasvir / sofosbuvir',
 'symbyax',
 'tamsulosin',
 'doxycycline',
 'dulaglutide',
 'intuniv',
 'buprenorphine',
 'qvar',
 'opdivo',
 'pyridium',
 'latuda',
 'bupropion',
 'implanon',
 'effexor xr',
 'drospirenone / ethinyl estradiol',
 'nuvaring',
 'prepopik',
 'tretinoin',
 'gildess fe 1 / 20',
 'ethinyl estradiol / norgestimate'

In [20]:
df_uci['review']

0         "It has no side effect, I take it in combinati...
1         "My son is halfway through his fourth week of ...
2         "I used to take another oral contraceptive, wh...
3         "This is my first time using any form of birth...
4         "Suboxone has completely turned my life around...
                                ...                        
161292    "I wrote my first report in Mid-October of 201...
161293    "I was given this in IV before surgey. I immed...
161294    "Limited improvement after 4 months, developed...
161295    "I&#039;ve been on thyroid medication 49 years...
161296    "I&#039;ve had chronic constipation all my adu...
Name: review, Length: 161297, dtype: object

In [21]:
count = 0

'''
TRAIN_DATA = [
    ('Who is Nishanth?', {'entities': [(7, 15, 'PERSON')]})
]
'''

TRAIN_DATA = []
d = 0
for _, item in df_uci.iterrows():
    ent_dict = {}
    if count < 5000:
        # print(_, item['drugName'])
        review = process_review(item['review'])
        #Locate drugs and their positions once and add to the visited items.
        visited_items = []
        #In case there is more than 1 entity 
        entities = [] 
        for token in review.split():
            if token in all_drugs_uci:
                for i in re.finditer(token, review):
                    if token not in visited_items:
                        # print(i[0], i.span()[0], i.span()[1])
                        entity = (i.span()[0], i.span()[1], 'CHEMICAL')
                        visited_items.append(token)
                        entities.append(entity)
                        d += 1
        if len(entities) > 0:
            ent_dict['entities'] = entities
            train_item = (review, ent_dict)
            TRAIN_DATA.append(train_item)
    else :
        break
    count+=1

In [22]:
TRAIN_DATA

[('it has no side effect i take it in combination of bystolic 5 mg and fish oil',
  {'entities': [(50, 58, 'CHEMICAL')]}),
 ('my son is halfway through his fourth week of intuniv we became concerned when he began this last week when he started taking the highest dose he will be on for two days he could hardly get out of bed was very cranky and slept for nearly 8 hours on a drive home from school vacation very unusual for him i called his doctor on monday morning and she said to stick it out a few days see how he did at school and with getting up in the morning the last two days have been problem free he is much more agreeable than ever he is less emotional a good thing less cranky he is remembering all the things he should overall his behavior is better we have tried many different medications and so far this is the most effective',
  {'entities': [(45, 52, 'CHEMICAL')]}),
 ('i used to take another oral contraceptive which had 21 pill cycle and was very happy very light periods max 5 d

In [23]:
import random
from spacy.training import Example

In [24]:
def train_spacy(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.blank("en")
    if "ner" not in nlp.pipe_names:
        # ner = nlp.create_pipe('ner')
        # nlp.add_pipe(ner, last=True)
        ner = nlp.add_pipe("ner")
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    
    optimizer = nlp.initialize()
    for itn in range(iterations):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for raw_text, entity_offsets in TRAIN_DATA:
            doc = nlp.make_doc(raw_text)
            example = Example.from_dict(doc, entity_offsets)
            nlp.update([example], sgd=optimizer, losses = losses)
            print(losses)
    return nlp



In [77]:
nlp2 = spacy.load("trained_model_5000")

In [26]:
displacy.render(nlp2(ex)[:], style='ent', jupyter=True)

In [27]:
import numpy as np
 
# function to get unique values
def unique(list1):
    x = np.array(list1)
    return np.unique(x)

In [28]:
alldrugs = []
alldrugs = set(alldrugs)
nlpo = nlp2(ex)

In [29]:
for name in nlpo.ents:
    if name.label_ == 'CHEMICAL':
        alldrugs.add(str(name))
nlpo = nlp(ex)
for name in nlpo.ents:
    if name.label_ == 'CHEMICAL':
        alldrugs.add(str(name))

alldrugs.add('Crocin')
alldrugs

{'Crocin', 'Nexplanon'}

In [30]:
'''Condition Finder If drug drug is recognized by nlp2'''
conditions = set()
rows = df_uci[df_uci['drugName'] == "Ortho Evra"]
for r in rows.iterrows():
    conditions.add(str(r[1][2]))
conditions

{'Birth Control'}

In [31]:
df_test = pd.read_csv("drugsComTest_raw.tsv", sep='\t')

In [84]:
test_data = []
count = 0
for _, item in df_test.iterrows():
    ent_dict = {}
    if count < 5000:
        # print(_, item['drugName'])
        review = process_review(item['review'])
        #Locate drugs and their positions once and add to the visited items.
        visited_items = []
        #In case there is more than 1 entity 
        entities = [] 
        for token in review.split():
            if token in all_drugs_uci:
                for i in re.finditer(token, review):
                    if token not in visited_items:
                        # print(i[0], i.span()[0], i.span()[1])
                        entity = (i.span()[0], i.span()[1], 'CHEMICAL')
                        visited_items.append(token)
                        entities.append(entity)
                        d += 1
        if len(entities) > 0:
            ent_dict['entities'] = entities
            train_item = (review, ent_dict)
            test_data.append(train_item)
    else :
        break
    count+=1

In [85]:
test_data

[('i039ve tried a few antidepressants over the years citalopram fluoxetine amitriptyline but none of those helped with my depression insomnia amp anxiety my doctor suggested and changed me onto 45mg mirtazapine and this medicine has saved my life thankfully i have had no side effects especially the most common  weight gain i039ve actually lost alot of weight i still have suicidal thoughts but mirtazapine has saved me',
  {'entities': [(50, 60, 'CHEMICAL'),
    (61, 71, 'CHEMICAL'),
    (72, 85, 'CHEMICAL'),
    (196, 207, 'CHEMICAL')]}),
 ('my son has crohn039s disease and has done very well on the asacol he has no complaints and shows no side effects he has taken as many as nine tablets per day at one time i039ve been very happy with the results reducing his bouts of diarrhea drastically',
  {'entities': [(59, 65, 'CHEMICAL')]}),
 ('contrave combines drugs that were used for alcohol smoking and opioid cessation people lose weight on it because it also helps control overeating i have n

In [ ]:
for c in test_data:
    print(c)

In [46]:
from spacy.scorer import Scorer
from spacy.tokens import Doc

In [144]:
invalid_span_tokens = re.compile(r'\s')
cleaned_data = []
for text, annotations in test_data:
    entities = annotations['entities']
    # print(entities)
    valid_entities = []
    for start, end, label in entities:
        # print(start, end, label)
        if len(valid_entities) == 0:
            valid_entities.append((start, end, label))
        else:
            ok = True
            for s, e, label in valid_entities:
                if (start >= s and start <= e) :
                    ok = False
            if ok :
                valid_entities.append((start, end, label))
    cleaned_data.append((text, {'entities': valid_entities}))
cleaned_data

[('i039ve tried a few antidepressants over the years citalopram fluoxetine amitriptyline but none of those helped with my depression insomnia amp anxiety my doctor suggested and changed me onto 45mg mirtazapine and this medicine has saved my life thankfully i have had no side effects especially the most common  weight gain i039ve actually lost alot of weight i still have suicidal thoughts but mirtazapine has saved me',
  {'entities': [(50, 60, 'CHEMICAL'),
    (61, 71, 'CHEMICAL'),
    (72, 85, 'CHEMICAL'),
    (196, 207, 'CHEMICAL')]}),
 ('my son has crohn039s disease and has done very well on the asacol he has no complaints and shows no side effects he has taken as many as nine tablets per day at one time i039ve been very happy with the results reducing his bouts of diarrhea drastically',
  {'entities': [(59, 65, 'CHEMICAL')]}),
 ('contrave combines drugs that were used for alcohol smoking and opioid cessation people lose weight on it because it also helps control overeating i have n

In [143]:
'''Train data(1st 5000 entries) against trained nlp2 Model'''
a = []
for text, annots in TRAIN_DATA:
    doc = nlp2.make_doc(text)
    a.append(Example.from_dict(doc, annots))
print(nlp2.evaluate(a))

c:\Users\RAZAK\Desktop\venv\venv\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "awful medicine the worst the side effects outweigh..." with entities "[(178, 183, 'CHEMICAL'), (342, 350, 'CHEMICAL')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\RAZAK\Desktop\venv\venv\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "many people are unreasonably scared of klonopin039..." with entities "[(39, 47, 'CHEMICAL')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\RAZAK\Desktop\venv\venv\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be ali

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.7642082429501085, 'ents_r': 0.8666666666666667, 'ents_f': 0.8122190201729107, 'ents_per_type': {'CHEMICAL': {'p': 0.7642082429501085, 'r': 0.8666666666666667, 'f': 0.8122190201729107}}, 'speed': 17233.527536132297}


In [146]:
'''Test Data (1st 5000 entries) against trained nlp2 model'''
a2 = []
for text, annots in cleaned_data:
    doc = nlp2.make_doc(text)
    a2.append(Example.from_dict(doc, annots))
print(nlp2.evaluate(a2))

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.7282930961457009, 'ents_r': 0.8211556829035339, 'ents_f': 0.7719416386083053, 'ents_per_type': {'CHEMICAL': {'p': 0.7282930961457009, 'r': 0.8211556829035339, 'f': 0.7719416386083053}}, 'speed': 17518.94063658834}


In [147]:
'''Test Data (1st 5000 entries) against Pretrained spaCy nlp model'''
a3 = []
for text, annots in cleaned_data:
    doc = nlp.make_doc(text)
    a3.append(Example.from_dict(doc, annots))
print(nlp.evaluate(a3))

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'tag_acc': None, 'pos_acc': None, 'morph_acc': None, 'morph_micro_p': None, 'morph_micro_r': None, 'morph_micro_f': None, 'morph_per_feat': None, 'lemma_acc': None, 'sents_p': None, 'sents_r': None, 'sents_f': None, 'dep_uas': None, 'dep_las': None, 'dep_las_per_type': None, 'ents_p': 0.206478482184174, 'ents_r': 0.5327125119388729, 'ents_f': 0.2976055492563196, 'ents_per_type': {'CHEMICAL': {'p': 0.5127556883475063, 'r': 0.5327125119388729, 'f': 0.5225436233751025}, 'DISEASE': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'speed': 4455.522762567055}
